In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/categories.json
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/ECG/image16.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/ECG/image59.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image91.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image96.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image39.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image69.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image23.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image61.jpeg
/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/Report/image94.jpeg


In [63]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os

%matplotlib inline

In [64]:
# Define data path
src_dir = '../input/healthcaredataimageclassification/Healthcare-images-classification'
ECG_dir = os.path.join(src_dir,'ECG')
Presciption_dir = os.path.join(src_dir,'Presciption')
Report_dir = os.path.join(src_dir,'Report')
Xray_dir = os.path.join(src_dir,'X-Ray')


In [65]:
print('ECG : ',len(os.listdir(ECG_dir)))
print('Presciption : ',len(os.listdir(Presciption_dir)))
print('Report : ',len(os.listdir(Report_dir)))
print('X-Ray : ',len(os.listdir(Xray_dir)))

ECG :  2
Presciption :  11
Report :  78
X-Ray :  10


In [66]:
generator = ImageDataGenerator(validation_split=0.4, rescale=1/255, shear_range=0.2, zoom_range=0.2, rotation_range=20, fill_mode='nearest')
train_data = generator.flow_from_directory(src_dir, batch_size=32, target_size=(150,150), subset='training')
val_data = generator.flow_from_directory(src_dir, batch_size=32,  target_size=(150,150), subset='validation')

Found 62 images belonging to 4 classes.
Found 39 images belonging to 4 classes.


In [67]:
import tensorflow_hub as hub
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5", 
                 output_shape=[2048],
                 trainable=False),
                 
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(4, activation='softmax')
])

'''
This TF Hub model uses the TF-Slim implementation of resnet_v2_50 with 50 layers.
The model contains a trained instance of the network, packaged to get feature vectors from images. 
'''

In [68]:
class Callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.90):
            print("\nReached %2.2f%% accuracy, training has been stop" %(logs.get('accuracy')*100))
            self.model.stop_training = True
callbacks = Callbacks()

In [69]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [70]:
historyModel = model.fit(
    train_data,
    steps_per_epoch = len(train_data),
    epochs = 20,
    validation_data = val_data,
    validation_steps = 5,
    verbose = 2,
    callbacks = [callbacks])

Epoch 1/20
2/2 - 12s - loss: 2.1082 - accuracy: 0.4677 - val_loss: 1.3436 - val_accuracy: 0.7949
Epoch 2/20
2/2 - 3s - loss: 1.4747 - accuracy: 0.7742
Epoch 3/20
2/2 - 3s - loss: 1.2914 - accuracy: 0.8548
Epoch 4/20
2/2 - 3s - loss: 0.7443 - accuracy: 0.8387
Epoch 5/20
2/2 - 3s - loss: 0.5459 - accuracy: 0.8710
Epoch 6/20
2/2 - 3s - loss: 0.4196 - accuracy: 0.8226
Epoch 7/20
2/2 - 3s - loss: 0.2564 - accuracy: 0.9032

Reached 90.32% accuracy, training has been stop


In [73]:
import json
import cv2
with open('../input/healthcaredataimageclassification/Healthcare-images-classification/categories.json', 'r') as f:
    cat_to_name = json.load(f)
    classes = list(cat_to_name.values())
    
IMAGE_SHAPE = (150, 150)

#function to load image
def load_image(filename):
    img = cv2.imread(os.path.join(filename))
    assert not isinstance(img,type(None)), 'image not found'
    
    img = cv2.resize(img, (IMAGE_SHAPE[0], IMAGE_SHAPE[1]) )
  
    img = img /255
    
    
    return img

#function to predict
def predict(image):
    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}


#actual predction
img = load_image('/kaggle/input/healthcaredataimageclassification/Healthcare-images-classification/X-Ray/image19.jpeg')
prediction = predict(img)
print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))

PREDICTED: class: X-Ray, confidence: 1.000000
